In [45]:
import csv
import numpy as np
import pandas as pd


def writer(name, a) :
    with open(('answ_'+str(name)+'.txt'), 'w') as w :
        w.write(str(a))

In [46]:
train = pd.read_csv('salary-train.csv.csv')
train.head()
# X = train.loc[:, 1:]
# y = train[0]

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [47]:
import re
train['FullDescription'] = train['FullDescription'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)
train


,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355
...,...,...,...,...
59995,as a result of continued growth first class s...,Whitley Bay,contract,26400
59996,php mvc web developer macclesfieldcirca ...,Macclesfield,permanent,26000
59997,staff nurse nursing home baldock white recru...,Baldock,NaN,24500
59998,this is one of the best agency side opportunit...,The City,permanent,65000


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(min_df=5)
X_train_text = vec.fit_transform(train["FullDescription"])

In [49]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

In [50]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_cat = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [51]:
from scipy.sparse import hstack
X_train = hstack([X_train_text, X_train_cat])
y_train = train["SalaryNormalized"]

In [52]:
from sklearn.linear_model import Ridge
est = Ridge(alpha=1, random_state=241)
est.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=241, solver='auto', tol=0.001)

In [54]:
test = pd.read_csv('salary-test-mini.csv.csv')
test['FullDescription'] = test['FullDescription'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)
test

X_test_text = vec.transform(test["FullDescription"])
X_test_text
# test['LocationNormalized'].fillna('nan', inplace=True)
# test['ContractTime'].fillna('nan', inplace=True)
X_test_cat = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test_text, X_test_cat])
y_pred = est.predict(X_test)

In [59]:
writer(101, y_pred.round(decimals=2))